In [ ]:
import time

import pandas as pd
import requests

# 1. 초기 설정
base_url = "http://apis.data.go.kr/1613000/RTMSDataSvcOffiRent/getRTMSDataSvcOffiRent"
service_key = requests.utils.unquote(
    "8afa4c269ca52320f0ba72cd7d98b8d8003a8c88cafcb41d001dddbca8debf89"
)

lawd_codes = {
    "서울(종로)": "11110",
    "충남(아산)": "44210",
    "전남(무안)": "46830",
    "전북(완주)": "45710",
    "충남(홍성)": "44800",
    "경북(안동)": "47170",
}

months = [f"2025{m:02d}" for m in range(6, 12)]  # 2025-06 ~ 2025-11
all_data = []

session = requests.Session()

for deal_ymd in months:
    print(f"{deal_ymd} 데이터 수집 시작...")

    for region_name, code in lawd_codes.items():
        page = 1
        total_added = 0

        while True:
            params = {
                "serviceKey": service_key,  # ✅ unquote 제거 권장
                "LAWD_CD": code,
                "DEAL_YMD": deal_ymd,
                "pageNo": page,
                "numOfRows": 1000,
                "_type": "json",
            }

            try:
                response = session.get(base_url, params=params, timeout=20)

                if response.status_code != 200:
                    print(
                        f"  서버 응답 에러: {response.status_code} ({region_name} {code})"
                    )
                    break

                data_dict = response.json()
                header = data_dict.get("response", {}).get("header", {})
                result_code = str(header.get("resultCode", "")).strip()
                result_msg = str(header.get("resultMsg", "")).strip()
                if result_code not in ("00", "000"):
                    print(
                        f"  ❌ API 에러 ({region_name} {code}, {deal_ymd}, page={page}): "
                        f"{result_code} / {result_msg}"
                    )
                    break

                body = data_dict.get("response", {}).get("body", {}) or {}
                items = body.get("items", {}) or {}
                item_list = items.get("item")

                # ✅ 더 이상 데이터 없으면 종료
                if not item_list:
                    if page == 1:
                        print(f"  [{deal_ymd}] {region_name}({code}): 데이터 없음")
                    break

                # ✅ 단건(dict) → list로 통일
                if isinstance(item_list, dict):
                    item_list = [item_list]

                # ✅ row마다 메타정보 추가(나중에 분석 편함)
                for it in item_list:
                    it["_deal_ymd"] = deal_ymd
                    it["_region_name"] = region_name
                    it["_lawd_cd"] = code
                    it["_pageNo"] = page

                all_data.extend(item_list)
                total_added += len(item_list)
                print(
                    f"페이지 {page}  [{deal_ymd}] 지역 {region_name}: {len(item_list)}건 추가"
                )

                # 다음 페이지로
                page += 1

                # API 서버 부하 방지
                time.sleep(0.2)

            except Exception as e:
                print(f"  [{deal_ymd}] {region_name}({code}) page={page} 오류: {e}")
                break

        if total_added > 0:
            print(
                f"  ✅ [{deal_ymd}] {region_name}({code}): 총 {total_added}건 수집 완료"
            )

print("-" * 30)
print(f"수집 종료: 총 {len(all_data)}건의 데이터가 리스트에 저장되었습니다.")


202506 데이터 수집 시작...
  [202506] 지역 서울(종로): 154건 추가
  ✅ [202506] 서울(종로)(11110): 총 154건 수집 완료
  [202506] 지역 충남(아산): 39건 추가
  ✅ [202506] 충남(아산)(44210): 총 39건 수집 완료
  [202506] 전남(무안)(46830): 데이터 없음
  [202506] 전북(완주)(45710): 데이터 없음
  [202506] 지역 충남(홍성): 7건 추가
  ✅ [202506] 충남(홍성)(44800): 총 7건 수집 완료
  [202506] 지역 경북(안동): 52건 추가
  ✅ [202506] 경북(안동)(47170): 총 52건 수집 완료
202507 데이터 수집 시작...
  [202507] 지역 서울(종로): 172건 추가
  ✅ [202507] 서울(종로)(11110): 총 172건 수집 완료
  [202507] 지역 충남(아산): 49건 추가
  ✅ [202507] 충남(아산)(44210): 총 49건 수집 완료
  [202507] 전남(무안)(46830): 데이터 없음
  [202507] 전북(완주)(45710): 데이터 없음
  [202507] 지역 충남(홍성): 12건 추가
  ✅ [202507] 충남(홍성)(44800): 총 12건 수집 완료
  [202507] 지역 경북(안동): 42건 추가
  ✅ [202507] 경북(안동)(47170): 총 42건 수집 완료
202508 데이터 수집 시작...
  [202508] 지역 서울(종로): 155건 추가
  ✅ [202508] 서울(종로)(11110): 총 155건 수집 완료
  [202508] 지역 충남(아산): 46건 추가
  ✅ [202508] 충남(아산)(44210): 총 46건 수집 완료
  [202508] 전남(무안)(46830): 데이터 없음
  [202508] 전북(완주)(45710): 데이터 없음
  [202508] 지역 충남(홍성): 8건 추가
  ✅ [202508] 충남(홍성)(

In [ ]:
df = pd.DataFrame(all_data)
print("df shape:", df.shape)
print(df.head())
